# Mediation Fallacy vs Total Effect
**Hands‑on Notebook**

This notebook illustrates the **mediation fallacy**:
adjusting for a mediator (income) when estimating the **total effect** of education on health.

Causal story:

- `edu` (education) → `income` → `health`
- We simulate data consistent with this DAG and compare:
  - A model that **does not adjust** for `income` (captures total effect)
  - A model that **adjusts** for `income` (captures only the direct effect)

This is a good companion to your slides on the mediation fallacy.

In [10]:
pip install kernel

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install statsmodels

In [15]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

rng = np.random.default_rng(0)
N = 100000

# Education (binary: 0 = low, 1 = high)
edu = rng.binomial(1, 0.5, size=N)

# Income = f(education) + noise
# Ground truth: each unit of edu increases income by 15,000
income = 20000 + 15000 * edu + rng.normal(0, 5000, size=N)

# Health score depends on both edu and income
# Direct effect of edu: 3
# Indirect effect via income: 0.0005 * 15000 = 7.5
# Total effect = 3 + 7.5 = 10.5
health = 50 + 3 * edu + 0.0005 * income + rng.normal(0, 5, size=N)

df = pd.DataFrame({"edu": edu, "income": income, "health": health})
df.head()

,edu,income,health
0,1,30775.862336,63.707910
1,0,23759.329162,57.618758
2,0,21352.320576,65.816718
3,0,19757.359858,53.559080
4,1,39192.971923,72.708255


## 1. Estimate the Total Effect (without adjusting for the mediator)

We first regress `health` on `edu` **without** adjusting for `income`.
This should recover (approximately) the **total causal effect** of education on health.

## Understanding Linear Regression Models

Before we analyze our data, let's understand the tool we'll use: **linear regression**.

### What is Linear Regression?

Linear regression helps us understand the relationship between variables. Think of it as fitting a straight line through data points to see how one variable (like education) affects another (like health).

### Ordinary Least Squares (OLS)

**Ordinary Least Squares (OLS)** is the most common technique for fitting a regression line to data. The idea is simple: among all possible lines we could draw, OLS chooses the one that makes our predictions most accurate. It does this by selecting the line where the total squared prediction errors are as small as possible.

Think of it this way: OLS picks the line that gets closest to all the data points simultaneously, minimizing how "wrong" our predictions are overall.

### The Formula

When you see `health ~ edu`, this means:
- **`health`** is what we're trying to predict (the outcome)
- **`edu`** is what we think influences health (the predictor)

The model estimates an equation like:
```
health = intercept + coefficient × edu + error
```

### What Does the Coefficient Mean?

The **coefficient** tells us: "When edu increases by 1 unit, health changes by this many units (on average)."

For example:
- If the coefficient for `edu` is 10.5, it means people with high education (`edu = 1`) have health scores about 10.5 points higher than those with low education (`edu = 0`)

### Why Multiple Models?

We'll run **two different models**:
1. One with just `edu` → shows the **total effect** of education on health
2. One with both `edu` and `income` → shows only the **direct effect** (not through income)

The difference between these reveals the mediation fallacy!

In [16]:
model_total = smf.ols("health ~ edu", data=df).fit()

# Extract just the coefficient for education
coef_total = model_total.params['edu']
print(f"Total effect of education on health: {coef_total:.2f}")
print(f"(Ground truth: 10.5)")

Total effect of education on health: 10.48
(Ground truth: 10.5)


### Interpreting the Results

The coefficient shows how much health scores differ between high and low education groups.

- **Ground truth total effect**: 10.5 points
- **Estimated total effect**: should be very close to 10.5 with our large sample

This captures the **full impact** of education on health, including both:
- The direct effect of education itself
- The indirect effect through increased income

## 2. Commit the Mediation Fallacy: Adjust for the Mediator

Now we **add `income` to the regression**:

```text
health ~ edu + income
```

This is equivalent to treating `income` as a confounder and **conditioning on the mediator**.
It will block the `edu → income → health` path.

In [17]:
model_adjusted = smf.ols("health ~ edu + income", data=df).fit()

# Extract just the coefficient for education
coef_adjusted = model_adjusted.params['edu']
print(f"Direct effect of education on health (adjusting for income): {coef_adjusted:.2f}")
print(f"(Ground truth: 3.0)")
print(f"\nDifference from total effect: {coef_total - coef_adjusted:.2f}")
print("This difference represents the indirect effect through income that we've blocked!")

Direct effect of education on health (adjusting for income): 2.96
(Ground truth: 3.0)

Difference from total effect: 7.52
This difference represents the indirect effect through income that we've blocked!


### The Mediation Fallacy Revealed

Compare the two coefficients:

- **Total effect** (without adjusting for income): ~10.5
- **Direct effect** (adjusting for income): ~3.0

By including `income` in the model, we've blocked the indirect path `edu → income → health`.

**This is the mediation fallacy**: when we adjust for a mediator while trying to estimate the total causal effect, we systematically **underestimate** that effect. In this case, we'd miss about 7.5 points of the true effect of education on health!

**Key takeaway**: Only adjust for mediators when you specifically want to isolate the direct effect. If you want the total effect, do NOT control for variables on the causal path.

## Exercise
Explain in words why stronger mediation makes the mediation fallacy more severe.

In [18]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

def simulate(mediation_strength):
    rng = np.random.default_rng(0)
    N = 100000
    edu = rng.binomial(1, 0.5, size=N)
    income = 20000 + mediation_strength * edu + rng.normal(0, 5000, size=N)
    health = 50 + 3 * edu + 0.0005 * income + rng.normal(0, 5, size=N)
    df = pd.DataFrame({"edu": edu, "income": income, "health": health})
    total = smf.ols("health ~ edu", data=df).fit().params['edu']
    direct = smf.ols("health ~ edu + income", data=df).fit().params['edu']
    return total, direct, total - direct

strengths = [5000, 15000, 30000, 60000]
results = [simulate(s) for s in strengths]

for s, (total, direct, diff) in zip(strengths, results):
    print(f"Mediation strength={s}: Total={total:.2f}, Direct={direct:.2f}, Gap={diff:.2f}")


Mediation strength=5000: Total=5.48, Direct=2.96, Gap=2.52
Mediation strength=15000: Total=10.48, Direct=2.96, Gap=7.52
Mediation strength=30000: Total=17.98, Direct=2.95, Gap=15.02
Mediation strength=60000: Total=32.98, Direct=2.95, Gap=30.03
